In [28]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from lime import lime_tabular

In [39]:
train = pd.read_csv ('https://raw.githubusercontent.com/claradelatorre/Test2ML-Classifier/main/train.csv')
test = pd.read_csv ('https://raw.githubusercontent.com/claradelatorre/Test2ML-Classifier/main/test.csv')

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        80000 non-null  int64  
 1   Customer_ID               80000 non-null  int64  
 2   Month                     80000 non-null  int64  
 3   Name                      80000 non-null  object 
 4   Age                       80000 non-null  float64
 5   SSN                       80000 non-null  float64
 6   Occupation                80000 non-null  object 
 7   Annual_Income             80000 non-null  float64
 8   Monthly_Inhand_Salary     80000 non-null  float64
 9   Num_Bank_Accounts         80000 non-null  float64
 10  Num_Credit_Card           80000 non-null  float64
 11  Interest_Rate             80000 non-null  float64
 12  Num_of_Loan               80000 non-null  float64
 13  Type_of_Loan              80000 non-null  object 
 14  Delay_

In [46]:
train.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,43161,3985,4,Lawrencez,40.0,856431180.0,Accountant,97384.44,8167.370000,1.0,...,Good,405.45,32.643281,336.0,No,77.284943,53.714219,High_spent_Large_value_payments,847.172909,Standard
1,40663,49494,2,Mohammedr,39.0,417453942.0,Writer,31369.58,2607.131667,8.0,...,Bad,4259.91,39.375507,144.0,Yes,171.748418,44.990811,High_spent_Large_value_payments,280.855383,Standard
2,111059,2925,6,Alistairn,17.0,499528888.0,Writer,30171.86,2772.321667,6.0,...,Bad,3406.51,34.039021,123.0,Yes,205.403926,40.781489,High_spent_Large_value_payments,247.322615,Standard
3,72139,21358,2,Dinesh Naire,22.0,859449371.0,Doctor,13100.02,1246.668333,7.0,...,Standard,727.29,34.747262,129.0,Yes,41.177132,23.005463,Low_spent_Large_value_payments,298.156832,Standard
4,119841,21221,4,Bakerg,26.0,485239890.0,Doctor,62313.54,5120.795000,6.0,...,Standard,817.64,24.545394,122.0,NM,196.679642,63.346750,Low_spent_Medium_value_payments,152.669902,Standard


In [48]:
#Eliminamos datos inútiles

train.drop(['ID','Customer_ID','Name','SSN'] ,axis= 1 , inplace = True)

In [50]:
#Convertimos datos - OrdinalEncoder

from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
for column in train.columns:
  if train[column].dtypes == 'object':
    train[column] = ordinal_encoder.fit_transform(train[[column]])


from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
for column in test.columns:
  if test[column].dtypes == 'object':
    test[column] = ordinal_encoder.fit_transform(test[[column]])

In [53]:
train.head()

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,4,40.0,0.0,97384.44,8167.370000,1.0,3.0,8.0,1.0,1410.0,...,1.0,405.45,32.643281,336.0,1.0,77.284943,53.714219,0.0,847.172909,2.0
1,2,39.0,14.0,31369.58,2607.131667,8.0,9.0,27.0,8.0,2528.0,...,0.0,4259.91,39.375507,144.0,2.0,171.748418,44.990811,0.0,280.855383,2.0
2,6,17.0,14.0,30171.86,2772.321667,6.0,7.0,30.0,9.0,5143.0,...,0.0,3406.51,34.039021,123.0,2.0,205.403926,40.781489,0.0,247.322615,2.0
3,2,22.0,3.0,13100.02,1246.668333,7.0,5.0,6.0,5.0,811.0,...,2.0,727.29,34.747262,129.0,2.0,41.177132,23.005463,3.0,298.156832,2.0
4,4,26.0,3.0,62313.54,5120.795000,6.0,2.0,18.0,4.0,520.0,...,2.0,817.64,24.545394,122.0,0.0,196.679642,63.346750,4.0,152.669902,2.0


In [52]:
#Separamos las variables.
#Variable objetivo:
y = train.iloc[:,-1]
y

0        2.0
1        2.0
2        2.0
3        2.0
4        2.0
        ... 
79995    2.0
79996    1.0
79997    2.0
79998    1.0
79999    0.0
Name: Credit_Score, Length: 80000, dtype: float64

In [54]:
#Demás variables:
x = train.iloc[:,0:24]
x

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,4,40.0,0.0,97384.44,8167.370000,1.0,3.0,8.0,1.0,1410.0,...,1.0,405.45,32.643281,336.0,1.0,77.284943,53.714219,0.0,847.172909,2.0
1,2,39.0,14.0,31369.58,2607.131667,8.0,9.0,27.0,8.0,2528.0,...,0.0,4259.91,39.375507,144.0,2.0,171.748418,44.990811,0.0,280.855383,2.0
2,6,17.0,14.0,30171.86,2772.321667,6.0,7.0,30.0,9.0,5143.0,...,0.0,3406.51,34.039021,123.0,2.0,205.403926,40.781489,0.0,247.322615,2.0
3,2,22.0,3.0,13100.02,1246.668333,7.0,5.0,6.0,5.0,811.0,...,2.0,727.29,34.747262,129.0,2.0,41.177132,23.005463,3.0,298.156832,2.0
4,4,26.0,3.0,62313.54,5120.795000,6.0,2.0,18.0,4.0,520.0,...,2.0,817.64,24.545394,122.0,0.0,196.679642,63.346750,4.0,152.669902,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,7,50.0,10.0,107761.29,9047.107500,4.0,5.0,9.0,0.0,3463.0,...,2.0,34.09,41.899992,211.0,2.0,0.000000,150.220682,0.0,984.123010,2.0
79996,2,33.0,12.0,16218.32,1098.526667,6.0,6.0,21.0,5.0,3004.0,...,0.0,4466.33,24.275710,98.0,0.0,64.533114,34.457893,2.0,260.523798,1.0
79997,6,25.0,4.0,29374.77,2377.897500,3.0,7.0,6.0,2.0,6258.0,...,2.0,1189.37,38.439021,241.0,0.0,40.264074,22.234178,1.0,403.980355,2.0
79998,7,15.0,8.0,43782.45,3543.537500,6.0,9.0,25.0,8.0,4189.0,...,0.0,3468.25,24.233841,85.0,2.0,240.664739,42.839562,3.0,146.741586,1.0
